Handwritten digit recognition use SVM



In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K
from skimage.feature import hog
import cv2
import pickle
import gc
import joblib

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#Gọi dữ liệu có săn từ dataset mnist
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))
y = y.astype(int)
print("X Shape : ",X.shape)
print("y Shape : ",y.shape)

In [ ]:
# Hiển thị ra 10 ảnh đầu tiên trong dataset mnist
plt.figure(figsize=(15,7))
for i in range(10):
  plt.subplot(2,5,i+1)
  plt.imshow(X[i],cmap="gray")
  plt.axis("off")
  plt.title("X["+str(i)+"]")
plt.show()
# Nhãn của 10 ảnh đầu tiên trong dataset mnist
[label for label in y[:10]]

In [ ]:
#Gọi dữ liệu từ dataset drive
train = pd.read_csv('/content/drive/MyDrive/train.csv')
train.shape

In [ ]:
#Lấy dữ liệu, tách và reshape dữ liệu
X_drive = train.drop('label', axis = 1).values 
X_drive = X_drive.reshape(X_drive.shape[0],28,28)
y_drive = train.label.values
print("X_drive Shape : ",X_drive.shape)
print("y_drive Shape : ",y_drive.shape)

In [ ]:
# Gộp 2 tập dataset từ mnist và drive
X = np.concatenate((X, X_drive))
y = np.concatenate((y, y_drive))

print("X Shape : ",X.shape)
print("y Shape : ",y.shape)

In [ ]:
# Hiển thị 100 ảnh chữ số và label của nó 
numbers_to_display = 100
num_cells = math.ceil(math.sqrt(numbers_to_display))
plt.figure(figsize=(20,20))
for i in range(numbers_to_display):
    plt.subplot(num_cells, num_cells, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X[i], cmap=plt.cm.binary)
    plt.xlabel(y[i])
plt.show()

In [ ]:
X = X.reshape(X.shape[0],-1)
print("X Shape : ",X.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print("X_train Shape : ",X_train.shape)
print("y_train Shape : ",y_train.shape)
print("X_test Shape : ",X_test.shape)
print("y_test Shape : ",y_test.shape)

In [ ]:
label = sorted(pd.unique(y_train))
print(label)

In [ ]:
# Số lượng lớp và số lượng trong tập dữ liệu train
plt.plot(figure = (20,14))
g = sns.countplot(y_train, palette = 'icefire')
plt.title('Number of digit classes')
for i in range(10):
  print("Number "+str(i)+" :",y_train.tolist().count(i))

In [ ]:
# Số lượng lớp và số lượng trong tập dữ liệu test
plt.plot(figure = (20,14))
g = sns.countplot(y_test, palette = 'icefire')
plt.title('Number of digit classes')
for i in range(10):
  print("Number "+str(i)+" :",y_test.tolist().count(i))

Model Building

1. Linear SVM

In [ ]:
%%time
# Xây dựng mô hình tuyến tính 
model_linear = SVC(kernel='linear')
model_linear.fit(X_train, y_train)

# Dự đoán
y_pred_linear = model_linear.predict(X_test)

In [ ]:
filename = '/content/drive/MyDrive/model/model_svm_linear.sav'
pickle.dump(model_linear, open(filename, 'wb'))

In [ ]:
# ma trận và độ chính xác 
# Độ chính xác
print("Độ chính xác :", accuracy_score(y_true=y_test, y_pred=y_pred_linear), "\n")

# matrix
print("Ma trận độ chính xác : ")
print(confusion_matrix(y_true=y_test, y_pred=y_pred_linear))

# Độ chính xác theo lớp
print("\nĐộ chính xác theo lớp: ")
print(classification_report(y_test, y_pred_linear))

In [ ]:
plt.imshow(X_test[20].reshape(28,28))
model_linear.predict(X_test[20].reshape(1,-1))

In [ ]:
# Giải phóng bộ nhớ
gc.collect()

2. Non-Linear SVM with the RBF kernel.

In [ ]:
%%time
# sử dụng hạt nhân rbf, C = 1, giá trị mặc định của gamma

# xây dựng mô hình phi tuyến tính
model_linear_rbf = SVC(kernel='rbf')
model_linear_rbf.fit(X_train, y_train)

# predict
y_pred_rbf = model_linear_rbf.predict(X_test)

In [ ]:
filename = '/content/drive/MyDrive/model/model_svm_rbf.sav'
pickle.dump(model_linear_rbf, open(filename, 'wb'))

In [ ]:
# ma trận và độ chính xác 

# Độ chính xác
print("Độ chính xác :", accuracy_score(y_true=y_test, y_pred=y_pred_rbf), "\n")

# matrix
print("Ma trận độ chính xác : ")
print(confusion_matrix(y_true=y_test, y_pred=y_pred_rbf))

# Độ chính xác theo lớp
print("\nĐộ chính xác theo lớp: ")
print(classification_report(y_test, y_pred_rbf))

In [ ]:
plt.imshow(X_test[20].reshape(28,28))
model_linear_rbf.predict(X_test[20].reshape(1,-1))

Grid Search Cross-Validation

In [ ]:
%%time
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Tạo một đối tượng KFold với 5 phần chia
folds = KFold(n_splits = 5, shuffle = True, random_state = 10)

# chỉ định phạm vi siêu tham số
# Đặt tham số bằng xác thực chéo
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}

# Thiết lập GridSearchCV()
model_grid = GridSearchCV(estimator = SVC(), 
                        param_grid = param_grid, 
                        scoring= 'accuracy', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
model_grid.fit(X_train, y_train)

In [ ]:
# Kết quả cv
cv_results = pd.DataFrame(model_grid.cv_results_)
cv_results

In [ ]:
# chuyển đổi C thành kiểu số để vẽ trên trục x
cv_results['param_C'] = cv_results['param_C'].astype('int')

# plotting
plt.figure(figsize=(16,8))

# subplot 1/3
plt.subplot(131)
gamma_01 = cv_results[cv_results['param_gamma']==0.01]

plt.plot(gamma_01["param_C"], gamma_01["mean_test_score"])
plt.plot(gamma_01["param_C"], gamma_01["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.01")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')

# subplot 2/3
plt.subplot(132)
gamma_001 = cv_results[cv_results['param_gamma']==0.001]

plt.plot(gamma_001["param_C"], gamma_001["mean_test_score"])
plt.plot(gamma_001["param_C"], gamma_001["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.001")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')


# subplot 3/3
plt.subplot(133)
gamma_0001 = cv_results[cv_results['param_gamma']==0.0001]

plt.plot(gamma_0001["param_C"], gamma_0001["mean_test_score"])
plt.plot(gamma_0001["param_C"], gamma_0001["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.0001")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')

In [ ]:
# In điểm chính xác tối ưu và siêu tham số
best_score = model_grid.best_score_
best_hyperparams = model_grid.best_params_

print("Điểm kiểm tra tốt nhất là {0} tương ứng với siêu đường kính".format(best_score, best_hyperparams))

**Crop the image, extract features in the image**

In [ ]:
X_train = X_train.reshape(X_train.shape[0],28,28)
X_test = X_test.reshape(X_test.shape[0],28,28)
print("X Shape : ",X_train.shape)
print("X Shape : ",X_test.shape)

1. Step by step process of cropping and processing a photo

In [ ]:
img = X_test[1].astype(np.uint8)
img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
# Convert to grayscale and apply Gaussian filtering

# Use GaussianBlur
img_gray = cv2.GaussianBlur(img_gray, (5, 5), 0)

# Threshold the image
ret, im_th = cv2.threshold(img_gray, 80, 255, cv2.THRESH_BINARY )
# Find contours in the image
ctrs,_ = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
plt.imshow(im_th)

In [ ]:
boxes = []
for c in ctrs:
    (x, y, w, h) = cv2.boundingRect(c)
    boxes.append([x,y, x+w,y+h])

boxes = np.asarray(boxes)
left, top = np.min(boxes, axis=0)[:2]
right, bottom = np.max(boxes, axis=0)[2:]
left,right,top, bottom

In [ ]:
plt.imshow(im_th[top-1:bottom+1,left-1:right+1])

In [ ]:
roi = im_th[top-1:bottom+1,left-1:right+1]
# Resize the image
roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
roi = cv2.dilate(roi, (3, 3))
plt.imshow(roi)

2. Processing the training dataset

In [ ]:
list_ft = []
list_train_remove = []
i = -1
for img in X_train :
  i += 1
  img = img.astype(np.uint8)
  img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
  img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

  # Use GaussianBlur
  im_gray = cv2.GaussianBlur(img_gray, (3, 3), 0)

  # Threshold the image
  ret, im_th = cv2.threshold(im_gray, 100, 255, cv2.THRESH_BINARY)
  # Find contours in the image
  ctrs,_ = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Tìm khung chứa chữ số
  try:
    boxes = []
    for c in ctrs:
        (x, y, w, h) = cv2.boundingRect(c)
        boxes.append([x,y, x+w,y+h])
    boxes = np.asarray(boxes)
    left, top = np.min(boxes, axis=0)[:2]
    right, bottom = np.max(boxes, axis=0)[2:]

    roi = im_th[top-1:bottom+1,left-1:right+1]
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    roi = cv2.dilate(roi, (3, 3))
    # Calculate the HOG features
    roi = roi.reshape(-1)
    list_ft.append(roi)
  except Exception as e:
    list_train_remove.append(i)
    print(i,end=" - ")
features = np.array(list_ft, 'float32')

In [ ]:
features.shape

In [ ]:
labels = []
for i in range(y_train.shape[0]):
  if i not in list_train_remove :
    labels.append(y_train[i])
labels = np.array(labels)
print(labels[:10])
labels.shape

In [ ]:
# Hiển thị 100 ảnh chữ số và label của nó 
numbers_to_display = 100
num_cells = math.ceil(math.sqrt(numbers_to_display))
plt.figure(figsize=(20,20))
for i in range(numbers_to_display):
    plt.subplot(num_cells, num_cells, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(features[i].reshape(28,28))
    plt.xlabel(labels[i])
plt.show()

3. Training model

In [ ]:
# Training
model_cut_image = SVC()
model_cut_image.fit(features, labels)

In [ ]:
# Lưu model
joblib.dump(model_cut_image, "/content/drive/MyDrive/model/model_cut_image.sav", compress=3)

4. Test Model

In [ ]:
# Load the classifier
model_cut_image = joblib.load("/content/drive/MyDrive/model/model_cut_image.sav")

In [ ]:
list_X_test = []
list_test_remove = []
i = -1
for img in X_test :
  i += 1
  img = img.astype(np.uint8)
  img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
  img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

  # Use GaussianBlur
  im_gray = cv2.GaussianBlur(img_gray, (3, 3), 0)

  # Threshold the image
  ret, im_th = cv2.threshold(im_gray, 100, 255, cv2.THRESH_BINARY)
  # Find contours in the image
  ctrs,_ = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Tìm khung chứa chữ số
  try:
    boxes = []
    for c in ctrs:
        (x, y, w, h) = cv2.boundingRect(c)
        boxes.append([x,y, x+w,y+h])
    boxes = np.asarray(boxes)
    left, top = np.min(boxes, axis=0)[:2]
    right, bottom = np.max(boxes, axis=0)[2:]

    roi = im_th[top-1:bottom+1,left-1:right+1]
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    roi = cv2.dilate(roi, (3, 3))
    # Calculate the HOG features
    roi = roi.reshape(-1)
    list_X_test.append(roi)
  except Exception as e:
    list_test_remove.append(i)
    print(i,end=" - ")
features_test = np.array(list_X_test, 'float32')

In [ ]:
labels_test = []
for i in range(y_test.shape[0]):
  if i not in list_test_remove :
    labels_test.append(y_test[i])
labels_test = np.array(labels_test)
print(labels_test[:10])
print(features_test.shape)
print(labels_test.shape)

In [ ]:
# ma trận và độ chính xác 
labels_pred = model_cut_image.predict(features_test)
# Độ chính xác
print("Độ chính xác :", accuracy_score(y_true=labels_test, y_pred=labels_pred), "\n")

# matrix
print("Ma trận độ chính xác : ")
print(confusion_matrix(y_true=labels_test, y_pred=labels_pred))

# Độ chính xác theo lớp
print("\nĐộ chính xác theo lớp: ")
print(classification_report(labels_test, labels_pred))

In [ ]:
# Hiển thị 100 ảnh chữ số và label của nó 
numbers_to_display = 100
num_cells = math.ceil(math.sqrt(numbers_to_display))
plt.figure(figsize=(20,20))
for i in range(numbers_to_display):
    plt.subplot(num_cells, num_cells, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(features_test[i].reshape(28,28))
    plt.xlabel("Predict : "+str(labels_pred[i]))
plt.show()